In [1]:
import sys
import os

# Add the project root to PYTHONPATH
root_path = os.path.abspath("..")    # Notebook → parent folder
sys.path.append(root_path)

from src.agent import DDQNAgent
from src.evaluation import run_episode
import json
from pathlib import Path
import imageio
import gymnasium as gym

In [2]:
experiment="20251123-095907"

# Fixed path to the agent folder (adjust if needed)
agent_dir = "/home/jaumemanero/_DEVELOPMENT/_Notebooks/AGENTS-course/DDQN-project/logs/experiment_04/"+experiment+"/agent/"

In [5]:


# Load agent metadata to rebuild the model
with open(agent_dir+"ddqn_agent.meta.json", "r") as f:
    meta = json.load(f)

obs_shape = tuple(meta["observation_shape"])
action_size = int(meta["action_size"])
agent = DDQNAgent(
    observation_shape=obs_shape,
    action_size=action_size,
    hidden_sizes=tuple(meta.get("hidden_sizes", (128, 128))),
    lr=float(meta.get("lr", 1e-3)),
    gamma=float(meta.get("gamma", 0.99)),
    replay_capacity=int(meta.get("replay_capacity", 10000)),
)
agent.load(str(agent_dir+"ddqn_agent"))

# Read env_name from run.meta.json, fallback to LunarLander-v3
env_name = "LunarLander-v3"
run_meta_path = Path(agent_dir + "run.meta.json")
if run_meta_path.exists():
    with open(run_meta_path, "r") as f:
        run_meta = json.load(f)
    env_name = run_meta.get("config", {}).get("env_name", env_name)

# Generate one episode video
env = gym.make(env_name, render_mode="rgb_array")
total, length, frames = run_episode(env, agent, render=True, max_steps=1000)
env.close()

out_path = "episode.mp4"
imageio.mimwrite(out_path, frames, fps=30)

print(f"Saved video to: {out_path}")
print(f"Episode reward: {total:.2f}, length: {length}")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Saved video to: episode.mp4
Episode reward: 205.45, length: 757


In [6]:
from IPython.display import Video

# Show the generated video
Video("episode.mp4", embed=True)

In [8]:
with open(agent_dir+"ddqn_agent.meta.json", "r") as f:
   data = json.load(f)

print(json.dumps(data, indent=4))

{
    "observation_shape": [
        8
    ],
    "action_size": 4,
    "epsilon": 0.01,
    "epsilon_final": 0.01,
    "epsilon_decay": 0.0001,
    "hidden_sizes": [
        128,
        128
    ],
    "lr": 0.0004,
    "gamma": 0.99,
    "replay_capacity": 100000,
    "network_type": "mlp",
    "torch_available": true
}
